This example demonstrates how to apply NAGL charges to a protein-ligand complex without anything more (relatively) complicated.

This lifts heavily from the existing [BRD4 benchmark example](https://docs.openforcefield.org/en/latest/examples/openforcefield/openff-toolkit/using_smirnoff_with_amber_protein_forcefield/BRD4_inhibitor_benchmark.html) from the toolkit.

In [20]:
import requests

repo_url = (
    "https://raw.githubusercontent.com/MobleyLab/benchmarksets/master/input_files/"
)
sources = {
    "receptor.pdb": repo_url + "BRD4/pdb/BRD4.pdb",
    "ligand.pdb": repo_url + "BRD4/pdb/ligand-1.pdb",
    "ligand.sdf": repo_url + "BRD4/sdf/ligand-1.sdf",
}
for filename, url in sources.items():
    r = requests.get(url)
    open(filename, "w").write(r.text)


In [21]:
from openff.toolkit import ForceField, Molecule, Topology
from openff.toolkit.utils.toolkits import NAGLToolkitWrapper, toolkit_registry_manager, ToolkitRegistry, RDKitToolkitWrapper

In [22]:
# Load the protein from PDB into a Topology object
topology = Topology.from_pdb("receptor.pdb")

# split the protein off into a distinct Molecule object for isolated charge assignment
protein = topology.molecule(0)

# load the already-docked ligand and add it to the topology
topology.add_molecules([Molecule.from_file("ligand.sdf")])

[1]

In [23]:
# need to provide both NAGL (for GCNN-based charge assignment) and RDKit (ONLY for SMARTS matching used by NAGL) wrapper
with toolkit_registry_manager(
    ToolkitRegistry([
        NAGLToolkitWrapper(), RDKitToolkitWrapper(),])):
    protein.assign_partial_charges(
        partial_charge_method="openff-gnn-am1bcc-0.1.0-rc.3.pt",
    )

[12:22:45] ERROR: Too many atoms [did you forget 'LargeMolecules' switch?]



In [24]:
# Load the SMIRNOFF port of ff14SB (whose charges won't be used) and Sage for protein and ligand, respectively
ff14sb_and_sage = ForceField("openff-2.2.1.offxml", "ff14sb_off_impropers_0.0.3.offxml")

interchange = ff14sb_and_sage.create_interchange(topology, charge_from_molecules=[protein])

In [25]:
interchange.visualize()

NGLWidget()

From here, you can prepare OpenMM, GROMACS, Amber, etc. simulations. See the [Interchange docs](https://docs.openforcefield.org/projects/interchange/en/stable/using/output.html) for more info.